In [1]:
import os

# Use the eager mode
os.environ['PT_HPU_LAZY_MODE'] = '0'

# Verify the environment variable is set
print(f"PT_HPU_LAZY_MODE: {os.environ['PT_HPU_LAZY_MODE']}")

import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import habana_frameworks.torch.core as htcore

PT_HPU_LAZY_MODE: 0


Calling add_step_closure function does not have any effect. It's lazy mode only functionality. (warning logged once)
Calling mark_step function does not have any effect. It's lazy mode only functionality. (warning logged once)
Calling iter_mark_step function does not have any effect. It's lazy mode only functionality. (warning logged once)


2.4.0a0+git74cd574


/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)


In [2]:
import os.path as osp
import numpy as np
import torch
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec

# MetaPath2Vec

[paper](https://ericdongyx.github.io/papers/KDD17-dong-chawla-swami-metapath2vec.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/metapath2vec.py)

In [3]:
device = torch.device("hpu")

In [4]:
# load the dataset
path = osp.join('.', 'data', 'AMiner')
dataset = AMiner(path)
data = dataset[0]

data = data.to(device)

============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 0
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
 PT_HPU_EAGER_PIPELINE_ENABLE = 1
 PT_HPU_EAGER_COLLECTIVE_PIPELINE_ENABLE = 1
---------------------------: System Configuration :---------------------------
Num CPU Cores : 160
CPU RAM       : 2113407800 KB
------------------------------------------------------------------------------


In [5]:
print(data)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)


In [6]:
print(type(data.edge_index_dict))
print(data.edge_index_dict[('paper', 'written_by', 'author')])

<class 'dict'>
tensor([[      0,       1,       2,  ..., 3194404, 3194404, 3194404],
        [      0,       1,       2,  ...,    4393,   21681,  317436]],
       device='hpu:0')


In [7]:
data.edge_index_dict.keys()

dict_keys([('paper', 'written_by', 'author'), ('author', 'writes', 'paper'), ('paper', 'published_in', 'venue'), ('venue', 'publishes', 'paper')])

In [8]:
print(type(data.num_nodes_dict))
print(data.num_nodes_dict)

<class 'dict'>
{'author': 1693531, 'venue': 3883, 'paper': 3194405}


In [9]:
print(type(data.y_dict))
print(data.y_dict["venue"])

<class 'dict'>
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], device='hpu:0')


In [10]:
print(type(data.y_index_dict))
print(data.y_index_dict["venue"])

<class 'dict'>
tensor([1741, 2245,  111,  837, 2588, 2116, 2696, 3648, 3784,  313, 3414,  598,
        2995, 2716, 1423,  783, 1902, 3132, 1753, 2748, 2660, 3182,  775, 3339,
        1601, 3589,  156, 1145,  692, 3048,  925, 1587,  820, 1374, 3719,  819,
         492, 3830, 2777, 3001, 3693,  517, 1808, 2353, 3499, 1763, 2372, 1030,
         721, 2680, 3355, 1217, 3400, 1271, 1970, 1127,  407,  353, 1471, 1095,
         477, 3701,   65, 1009, 1899, 1442, 2073, 3143, 2466,  289, 1996, 1070,
        3871, 3695,  281, 3633,   50, 2642, 1925, 1285, 2587, 3814, 3582, 1873,
        1339, 3450,  271, 2966,  453, 2638, 1354, 3211,  391, 1588, 3875, 2216,
        2146, 3765, 2486,  661, 3367,  426,  750, 2158,  519,  230, 1677,  839,
        2945, 1313, 1037, 2879, 2225, 3523, 1247,  448,  227, 3385,  529, 2849,
        1584, 1229,  373, 2235, 1819, 1764, 3155, 2852, 2789, 3474, 1571, 2088,
         208,  462], device='hpu:0')


In [11]:
# define the model

# dict_keys([('paper', 'written_by', 'author'), ('author', 'writes', 'paper'), ('paper', 'published_in', 'venue'), ('venue', 'publishes', 'paper')])

metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper'),
    ('paper', 'written_by', 'author'),
]


model = MetaPath2Vec(data.edge_index_dict, 
                     embedding_dim=128,
                     metapath=metapath,
                     walk_length=5, 
                     context_size=3,
                     walks_per_node=3,
                     num_negative_samples=1,
                    #  sparse=True,
                     sparse=False,
                    ).to(device)


In [12]:
# use the loader to build a loader
loader = model.loader(batch_size=128, shuffle=True, num_workers=3)

In [13]:
for idx, (pos_rw, neg_rw) in enumerate(loader):
    if idx == 10: break
    print(idx, pos_rw.shape, neg_rw.shape)

0 torch.Size([1536, 3]) torch.Size([1536, 3])
1 torch.Size([1536, 3]) torch.Size([1536, 3])
2 torch.Size([1536, 3]) torch.Size([1536, 3])
3 torch.Size([1536, 3]) torch.Size([1536, 3])
4 torch.Size([1536, 3]) torch.Size([1536, 3])
5 torch.Size([1536, 3]) torch.Size([1536, 3])
6 torch.Size([1536, 3]) torch.Size([1536, 3])
7 torch.Size([1536, 3]) torch.Size([1536, 3])
8 torch.Size([1536, 3]) torch.Size([1536, 3])
9 torch.Size([1536, 3]) torch.Size([1536, 3])


In [14]:
print(pos_rw[0],neg_rw[0])

tensor([1639018, 4891819, 4891819]) tensor([1639018, 4741022, 4891387])


In [15]:
# Inizialize optimizer
# optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
optimizer = torch.optim.Adam(list(model.parameters()), lr=0.01)

In [16]:
def train(epoch, log_steps=500, eval_steps=1000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))

@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('author', batch=data.y_index_dict['author'])
    y = data.y_dict['author']

    perm = torch.randperm(z.size(0), device=z.device)
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm],
                      y[test_perm], max_iter=150)


In [17]:
model.train()
model = torch.compile(model, backend="hpu_backend")
for epoch in range(1, 2):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

Epoch: 1, Step: 00500/13231, Loss: 6.4062
Epoch: 1, Step: 01000/13231, Loss: 6.4185


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


Epoch: 1, Step: 01000/13231, Acc: 0.2770
Epoch: 1, Step: 01500/13231, Loss: 6.4084
Epoch: 1, Step: 02000/13231, Loss: 6.4046
Epoch: 1, Step: 02000/13231, Acc: 0.2760
Epoch: 1, Step: 02500/13231, Loss: 6.4046
Epoch: 1, Step: 03000/13231, Loss: 6.4007
Epoch: 1, Step: 03000/13231, Acc: 0.2757
Epoch: 1, Step: 03500/13231, Loss: 6.4095
Epoch: 1, Step: 04000/13231, Loss: 6.4061
Epoch: 1, Step: 04000/13231, Acc: 0.2759
Epoch: 1, Step: 04500/13231, Loss: 6.4176
Epoch: 1, Step: 05000/13231, Loss: 6.4154
Epoch: 1, Step: 05000/13231, Acc: 0.2762
Epoch: 1, Step: 05500/13231, Loss: 6.4110
Epoch: 1, Step: 06000/13231, Loss: 6.4030
Epoch: 1, Step: 06000/13231, Acc: 0.2748
Epoch: 1, Step: 06500/13231, Loss: 6.4308
Epoch: 1, Step: 07000/13231, Loss: 6.4115
Epoch: 1, Step: 07000/13231, Acc: 0.2768
Epoch: 1, Step: 07500/13231, Loss: 6.3976
Epoch: 1, Step: 08000/13231, Loss: 6.4083
Epoch: 1, Step: 08000/13231, Acc: 0.2759
Epoch: 1, Step: 08500/13231, Loss: 6.4126
Epoch: 1, Step: 09000/13231, Loss: 6.4040


In [24]:
# save the model to "mymodel"

torch.save(model.state_dict(), "mymodel")

In [26]:
model.state_dict()

OrderedDict([('_orig_mod.embedding.weight',
              tensor([[ 1.0630,  0.1488, -0.1911,  ...,  1.5862,  1.5311, -2.1510],
                      [ 0.1243,  1.2492,  1.2393,  ...,  0.2227,  2.1340, -0.1600],
                      [ 1.4810,  0.7495,  0.4144,  ..., -1.2538,  1.9429,  0.6207],
                      ...,
                      [-1.3474, -0.3566, -1.2574,  ...,  0.4756, -0.4529, -0.1462],
                      [-0.9339, -0.2493, -1.7788,  ..., -0.0246,  1.1848,  0.1878],
                      [ 1.0832, -0.6992,  0.2164,  ..., -2.0589,  1.1881,  0.4817]],
                     device='hpu:0'))])

# load the model

In [21]:
loaded_model = MetaPath2Vec(data.edge_index_dict, 
                     embedding_dim=128,
                     metapath=metapath,
                     walk_length=5, 
                     context_size=3,
                     walks_per_node=3,
                     num_negative_samples=1,
                    #  sparse=True,
                     sparse=False,
                    ).to(device)

In [22]:
print(loaded_model.embedding.weight[1][:5])

tensor([1.0402, 0.4621, 0.4585, 0.4498, 0.5106], device='hpu:0',
       grad_fn=<SliceBackward0>)


In [27]:
loaded_file = torch.load("mymodel")
print(loaded_file)

/tmp/ipykernel_2511787/261134530.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_file = torch.load("mymodel")


OrderedDict([('_orig_mod.embedding.weight', tensor([[ 1.0630,  0.1488, -0.1911,  ...,  1.5862,  1.5311, -2.1510],
        [ 0.1243,  1.2492,  1.2393,  ...,  0.2227,  2.1340, -0.1600],
        [ 1.4810,  0.7495,  0.4144,  ..., -1.2538,  1.9429,  0.6207],
        ...,
        [-1.3474, -0.3566, -1.2574,  ...,  0.4756, -0.4529, -0.1462],
        [-0.9339, -0.2493, -1.7788,  ..., -0.0246,  1.1848,  0.1878],
        [ 1.0832, -0.6992,  0.2164,  ..., -2.0589,  1.1881,  0.4817]],
       device='hpu:0'))])


In [28]:
loaded_file['_orig_mod.embedding.weight']

tensor([[ 1.0630,  0.1488, -0.1911,  ...,  1.5862,  1.5311, -2.1510],
        [ 0.1243,  1.2492,  1.2393,  ...,  0.2227,  2.1340, -0.1600],
        [ 1.4810,  0.7495,  0.4144,  ..., -1.2538,  1.9429,  0.6207],
        ...,
        [-1.3474, -0.3566, -1.2574,  ...,  0.4756, -0.4529, -0.1462],
        [-0.9339, -0.2493, -1.7788,  ..., -0.0246,  1.1848,  0.1878],
        [ 1.0832, -0.6992,  0.2164,  ..., -2.0589,  1.1881,  0.4817]],
       device='hpu:0')

In [30]:
# load the model
# loaded_model.load_state_dict(torch.load("mymodel").detach().cpu())
loaded_model.embedding.weight = torch.nn.Parameter(loaded_file['_orig_mod.embedding.weight'])

In [31]:
print(loaded_model.embedding.weight[1][:5])

tensor([ 0.1243,  1.2492,  1.2393,  0.1696, -0.9190], device='hpu:0',
       grad_fn=<SliceBackward0>)


In [33]:
z_venue = loaded_model('venue', batch=data.y_index_dict['venue']).cpu().detach().numpy()
z_auth = loaded_model('author', batch=data.y_index_dict['author']).cpu().detach().numpy()

In [34]:
z_venue = z_venue[0:100]
z_auth = z_auth[0:100]

In [ ]:
!pip install umap-learn

In [37]:
data

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)

In [38]:
data

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)